In [ ]:
import numpy as np

In [ ]:
import qiskit.pulse as pulse

In [ ]:
from qiskit.circuit import Parameter

In [ ]:
from qiskit_experiments.calibration_management.backend_calibrations import BackendCalibrations

In [ ]:
from qiskit import IBMQ, schedule

In [ ]:
IBMQ.load_account()

In [ ]:
provider=IBMQ.get_provider(hub='ibm-q', group='open', project='main')

In [ ]:
backend=provider.get_backend("ibmq_armonk")

In [ ]:
backend.status()

In [ ]:
qubit=0

In [ ]:
def setup_cals(backend) -> BackendCalibrations:
    """A function to instantiate calibrations and add a couple of template schedules."""
    cals = BackendCalibrations(backend)

    dur = Parameter("dur")
    amp = Parameter("amp")
    sigma = Parameter("σ")
    beta = Parameter("β")
    drive = pulse.DriveChannel(Parameter("ch0"))

    # Define and add template schedules.
    with pulse.build(name="xp") as xp:
        pulse.play(pulse.Drag(dur, amp, sigma, beta), drive)

    with pulse.build(name="xm") as xm:
        pulse.play(pulse.Drag(dur, -amp, sigma, beta), drive)

    with pulse.build(name="x90p") as x90p:
        pulse.play(pulse.Drag(dur, Parameter("amp"), sigma, Parameter("β")), drive)

    cals.add_schedule(xp)
    cals.add_schedule(xm)
    cals.add_schedule(x90p)

    return cals

def add_parameter_guesses(cals: BackendCalibrations):
    """Add guesses for the parameter values to the calibrations."""
    for sched in ["xp", "x90p"]:
        cals.add_parameter_value(80, "σ", schedule=sched)
        cals.add_parameter_value(0.5, "β", schedule=sched)
        cals.add_parameter_value(320, "dur", schedule=sched)
        cals.add_parameter_value(0.5, "amp", schedule=sched)


In [ ]:
cals = setup_cals(backend)
add_parameter_guesses(cals)

In [ ]:
from qiskit_experiments.calibration_management.basis_gate_library import FixedFrequencyTransmon

In [ ]:
library = FixedFrequencyTransmon(default_values={"duration": 320})
cals = BackendCalibrations(backend, library)

In [ ]:
from qiskit_experiments.library.characterization.qubit_spectroscopy import QubitSpectroscopy

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(cals.parameters_table(qubit_list=[qubit, ()]))

In [ ]:
freq01_estimate = backend.defaults().qubit_freq_est[qubit]
frequencies = np.linspace(freq01_estimate -15e6, freq01_estimate + 15e6, 51)
spec = QubitSpectroscopy(qubit, frequencies)
spec.set_experiment_options(amp=0.1)

In [ ]:
circuit = spec.circuits(backend)[0]
circuit.draw(output="mpl");

In [ ]:
schedule(circuit, backend).draw();

In [ ]:
spec_data = spec.run(backend)

In [ ]:
spec_data.block_for_results()